In [12]:
from gurobipy import Model, GRB, GurobiError
from graph_tool.generation import complete_graph
from graph_tool.flow import boykov_kolmogorov_max_flow, min_st_cut
import numpy as np

In [8]:
n = 50
V = range(n)
xs = np.random.uniform(low=0, high=100, size=n)
ys = np.random.uniform(low=0, high=100, size=n)
cost = {
    (i, j): (xs[i] - xs[j]) ** 2 + (ys[i] - ys[j]) ** 2
    for i in V for j in V if i != j
}
A = list(cost.keys())

In [9]:
G = complete_graph(N=n, self_loops=False, directed=True)
cap = G.new_edge_property(value_type='double')
G.edge_properties['cap'] = cap

In [11]:
m = Model()
x = m.addVars(A, vtype=GRB.BINARY, obj=cost, name='x')
m.addConstrs(x.sum(i, '*') == 1 for i in V)
m.addConstrs(x.sum('*', i) == 1 for i in V)
m.setParam(GRB.Param.LazyConstraints, 1)

Set parameter LazyConstraints to value 1


In [13]:
def set_capacity():
    for e in G.edges():
        i, j = e.source(), e.target()

        try:
            xval = m.cbGetSolution(x[i,j])
        except GurobiError:
            xval = m.cbGetNodeRel(x[i,j])

        cap[e] = xval

In [14]:
def add_sec_for(subtour):
    m.cbLazy(sum(x[i,j] for i,j in A if i in subtour and j not in subtour) >= 1)

In [15]:
def callback(what, where):
    if where not in (GRB.Callback.MIPSOL, GRB.Callback.MIPNODE):
        return
    
    if where == GRB.Callback.MIPNODE and m.cbGet(GRB.Callback.MIPNODE_STATUS) != GRB.OPTIMAL:
        return
    
    set_capacity()

    source = G.vertex(0)
    added  = set()

    for i in range(1, n):
        if i in added:
            continue

        sink = G.vertex(i)
        res = boykov_kolmogorov_max_flow(g=G, source=source, target=sink, capacity=cap)

        # Create an edge property map quickly by
        # copying an existing one.
        flow = res.copy()

        # The value held by the property map is in
        # the .a member. Because capacity == flow + residuals
        # we must write:
        flow.a = cap.a - res.a

        maxflow = sum(flow[a] for a in sink.in_edges())

        if maxflow < 1 - 1e-6:
            print(f"Violated SEC. Flow = {maxflow:.3f} < 1")
            cut = min_st_cut(g=G, source=source, capacity=cap, residual=res)

            assert cut[source] == True
            assert cut[sink] == False

            subtour = [j for j in V if cut[G.vertex(j)] == False]

            assert len(subtour) < n

            add_sec_for(subtour)

            added = added.union(subtour)

In [16]:
m.optimize(callback=callback)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Fedora Linux 39 (KDE Plasma)")

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 100 rows, 2450 columns and 4900 nonzeros
Model fingerprint: 0x7f9fdd5e
Variable types: 0 continuous, 2450 integer (2450 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 148005.03092
Presolve time: 0.02s
Presolved: 100 rows, 2450 columns, 4900 nonzeros
Variable types: 0 continuous, 2450 integer (2450 binary)

Root relaxation: objective 5.375171e+03, 82 iterations, 0.00 seconds (0.00 work units)
Violated SEC. Flow = 0.000 < 1

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It